In [1]:
from preprocessing_class import *

In [2]:
train = pd.read_csv('C:/lg_capstone/find_abnormal/data/train_begin.csv')
test = pd.read_csv('C:/lg_capstone/find_abnormal/data/test_begin.csv')

preprocessing = Preprocessing(train, test)
preprocessing.dam()
preprocessing.fill1()
preprocessing.fill2()
preprocessing.autoclave()
preprocessing.final_preprocessing_v2()
X_train = preprocessing.X_train
y_train = preprocessing.y_train
X_val = preprocessing.X_val
y_val = preprocessing.y_val

X_test = preprocessing.X_test
Set_ID = preprocessing.Set_ID

submission_diff = preprocessing.submission_diff

c:\lg_capstone\find_abnormal\preprocessing_class.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_test['target'] = 'AbNormal'


preprocessing - dam
preprocessing - fill1
preprocessing - fill2


c:\lg_capstone\find_abnormal\preprocessing_class.py:396: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'].replace(304.8, 305.0, inplace=True) # 304.8을 305.0으로 대체
c:\lg_capstone\find_abnormal\preprocessing_class.py:397: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we

preprocessing - autoclave
Final preprocessing
Section 1
Section 2
Section 3


c:\lg_capstone\find_abnormal\preprocessing_class.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['target'] = le.fit_transform(train_data['target'].astype(str))
c:\lg_capstone\find_abnormal\preprocessing_class.py:663: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[column] = le.fit_transform(train_data[column].astype(str))
c:\lg_capstone\find_abnormal\preprocessing_class.py:663: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Section 4
Train data shape:  (40413, 73)
Features shape:  (40413, 69)
Section 5


In [3]:
# 1. SMOTE를 적용하여 학습한 랜덤 포레스트 모델
model_weighted_smote = RandomForestClassifier(n_estimators=200, random_state=42, class_weight={0: 1, 1: 10})
model_weighted_smote.fit(X_train, y_train)

# 2. 검증 데이터셋에서 예측 수행
y_pred_weighted_val = model_weighted_smote.predict(X_val)

# 3. 성능 평가
accuracy_weighted_val = accuracy_score(y_val, y_pred_weighted_val)
classification_rep_weighted_val = classification_report(y_val, y_pred_weighted_val, digits=4)

print("SMOTE 적용 모델 성능")
print("Accuracy:", accuracy_weighted_val)
print("Classification Report:\n", classification_rep_weighted_val)

# 4. 테스트 데이터에서 예측 수행
y_pred_test = model_weighted_smote.predict(X_test)

SMOTE 적용 모델 성능
Accuracy: 0.9396263763454163
Classification Report:
               precision    recall  f1-score   support

           0     0.2418    0.0499    0.0827       441
           1     0.9476    0.9910    0.9688      7642

    accuracy                         0.9396      8083
   macro avg     0.5947    0.5204    0.5257      8083
weighted avg     0.9091    0.9396    0.9204      8083



In [4]:
len(Set_ID), len(y_pred_test)

(17333, 17333)

In [5]:
# 6. 결과를 DataFrame으로 저장
result = pd.DataFrame({
    'Set ID': Set_ID.values,  # Set ID를 원래대로 유지
    'target': y_pred_test
})

# 결과 출력
print(result.head())

print(result.shape)

# 기존 0과 1 값을 'AbNormal'과 'Normal'로 변환
result['target'] = result['target'].map({0: 'AbNormal', 1: 'Normal'})

submission_same = result

                             Set ID  target
0  0001be084fbc4aaa9d921f39e595961b       1
1  0005bbd180064abd99e63f9ed3e1ac80       1
2  000948934c4140d883d670adcb609584       1
3  000a6bfd02874c6296dc7b2e9c5678a7       1
4  0018e78ce91343678716e2ea27a51c95       1
(17333, 2)


In [6]:
# submission_same, submission_diff 데이터프레임을 하나로 합치기
submission = pd.concat([submission_same, submission_diff])

In [7]:
submission.to_csv('C:/lg_capstone/find_abnormal/data/submission.csv', index=False)

In [8]:
unique, counts = np.unique(submission_same['target'], return_counts=True)
print(unique, counts)

['AbNormal' 'Normal'] [  169 17164]


In [ ]:
# submission_same, submission_diff 데이터프레임을 하나로 합치기
submission = pd.concat([submission_same, submission_diff])

In [ ]:
import pandas as pd
train_data = pd.read_csv('C:/lg_capstone/find_abnormal/data/train_begin.csv')

In [ ]:
# CURE START POSITION Z Collect Result_Fill2, CURE END POSITION Z Collect Result_Fill2 열만 추출
train_data = train_data[['CURE START POSITION Z Collect Result_Fill2', 'CURE END POSITION Z Collect Result_Fill2']]

In [ ]:
train_data['CURE START POSITION Z Collect Result_Fill2'].value_counts()

In [ ]:
# 'CURE START POSITION Z Collect Result_Fill2'가 33인 행만 추출
train_data_33_33 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 33 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 33)]
print(len(train_data_33_33))

train_data_33_32 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 33 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 32)]
print(len(train_data_33_32))

train_data_33_22 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 33 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 22)]
print(len(train_data_33_22))

In [ ]:
# 'CURE START POSITION Z Collect Result_Fill2'가 33인 행만 추출
train_data_start32 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 32]
train_data_start32['CURE END POSITION Z Collect Result_Fill2'].value_counts()

train_data_start32_32 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 32 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 32)]
print(len(train_data_start32_32))
train_data_start32_33 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 32 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 33)]
print(len(train_data_start32_33))

In [ ]:
# 'CURE START POSITION Z Collect Result_Fill2'가 33인 행만 추출
train_data_start22 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 22]
train_data_start22['CURE END POSITION Z Collect Result_Fill2'].value_counts()

train_data_start22_32 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 22 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 32)]
print(len(train_data_start22_32))
train_data_start22_22 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 22 & (train_data['CURE END POSITION Z Collect Result_Fill2'] == 22)]
print(len(train_data_start22_22))

In [ ]:
# 'CURE START POSITION Z Collect Result_Fill2'가 33인 행만 추출
train_data_start23 = train_data[train_data['CURE START POSITION Z Collect Result_Fill2'] == 23]
train_data_start23['CURE END POSITION Z Collect Result_Fill2'].value_counts()

print(train_data_start23['CURE END POSITION Z Collect Result_Fill2'].value_counts())
